In [58]:
import numpy as np
from sklearn import preprocessing

# Wczytanie danych
raw_csv_data = np.loadtxt('Audiobooks_data.csv', delimiter=',')

# Oddzielenie danych wejściowych i targetów
unscaled_inputs_all = raw_csv_data[:,1:-1]  # pomijamy ID
targets_all = raw_csv_data[:,-1]

# Przetasowanie danych
shuffled_indices = np.arange(unscaled_inputs_all.shape[0])
np.random.shuffle(shuffled_indices)

unscaled_inputs_all = unscaled_inputs_all[shuffled_indices]
targets_all = targets_all[shuffled_indices]

# Równoważenie zbioru danych (balansowanie targetów)
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

# Normalizacja danych wejściowych
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

# Przetasowanie po balansowaniu
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

# Podział na train/val/test
samples_count = shuffled_inputs.shape[0]
train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count + validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count + validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count + validation_samples_count:]
test_targets = shuffled_targets[train_samples_count + validation_samples_count:]

# Zapis do plików
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)


In [60]:
import numpy as np
import tensorflow as tf

# wyodrebnijmy dane wejsciowe za pomocą słowa kluczowego, pod którym je zapisaliśmy
npz = np.load('Audiobooks_data_train.npz')
train_inputs = npz['inputs'].astype(np.float32)
train_targets = npz['targets'].astype(np.int32)

# ładujemy dane walidacyjne do zmiennej tymczasowej
npz = np.load('Audiobooks_data_validation.npz')
validation_inputs = npz['inputs'].astype(np.float32)
validation_targets = npz['targets'].astype(np.int32)

# ładujemy dane testowe do zmiennej tymczasowej
npz = np.load('Audiobooks_data_test.npz')
test_inputs = npz['inputs'].astype(np.float32)
test_targets = npz['targets'].astype(np.int32)

# BUDOWA MODELU SIECI NEURONOWEJ
input_size = 10
output_size = 2
hidden_layer_size = 200

model = tf.keras.Sequential([
    # tf.keras.layers.Dense zasadniczo implementuje: output = activation(dot(input, weight) + bias)
    # przyjmuje kilka argumentów, ale dla nas najważniejsze są hidden_layer_size i funkcja activation
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),  # pierwsza ukryta warstwa
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),  # druga ukryta warstwa
    # ostatnia warstwa nie jest inna, użyj softmax
    tf.keras.layers.Dense(output_size, activation='softmax')  # warstwa wyjściowa
])

# ustawienie funkcji straty
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# TRENING MODELU
batch_size = 100
max_epochs = 200

# mechanizm wczesnego zatrzymywania
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

# i uczymy model
model.fit(train_inputs,                # dane wejściowe trenujące
          train_targets,               # targety wejściowe trenujące
          batch_size=batch_size,      # rozmiary podawanej partii
          epochs=max_epochs,          # ilość epok jakby wczesne zatrzymanie nie zadziałało
          callbacks=[early_stopping], # early_stopping zapobiega przetrenowaniu
          validation_data=(validation_inputs, validation_targets), # dane walidacyjne
          verbose=1                   # pokazanie postępów
)

# SPRAWDZAMY NASZ MODEL NA DANYCH TESTOWYCH, KTÓRYCH NIE WIDZIAŁ PODCZAS TRENINGU
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy * 100.))


Epoch 1/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.6866 - loss: 0.5774 - val_accuracy: 0.7808 - val_loss: 0.4181
Epoch 2/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7883 - loss: 0.4211 - val_accuracy: 0.8054 - val_loss: 0.3915
Epoch 3/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7956 - loss: 0.3932 - val_accuracy: 0.8076 - val_loss: 0.3700
Epoch 4/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8027 - loss: 0.3806 - val_accuracy: 0.8098 - val_loss: 0.3619
Epoch 5/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8048 - loss: 0.3719 - val_accuracy: 0.8166 - val_loss: 0.3530
Epoch 6/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8060 - loss: 0.3668 - val_accuracy: 0.8233 - val_loss: 0.3478
Epoch 7/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8090 - loss: 0.3629 - val_accuracy: 0.8210 - val_loss: 0.3452
Epoch 8/200
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8098 - loss: 0.3596 - val_accuracy: 0.8210 - 